source:
* https://docs.docker.com/network/
* https://docs.docker.com/network/network-tutorial-standalone/

This topic defines some basic Docker networking concepts and prepares you to design and deploy your applications to take full advantage of these capabilities.

Docker’s networking subsystem is pluggable, using drivers. Several drivers exist by default, and provide core networking functionality:

   **bridge** : The default network driver. If you don’t specify a driver, this is the type of network you are creating. Bridge networks are usually used when your applications run in standalone containers that need to communicate. 
   
   `Bridge network is the one I will use for my standalone applications`

**Bridge** : In terms of networking, a bridge network is a Link Layer device which forwards traffic between network segments. 

In terms of Docker, a bridge network uses a software bridge which allows containers connected to the same bridge network to communicate, while providing isolation from containers which are not connected to that bridge network. The Docker bridge driver automatically installs rules in the host machine so that containers on different bridge networks cannot communicate directly with each other.

Bridge networks apply to containers running on the same Docker daemon host. For communication among containers running on different Docker daemon hosts, you can either manage routing at the OS level, or you can use an overlay network.

Docker automatically creates a default bridge network and connects newly started dockers

`! docker network ls`

* NETWORK ID          NAME                DRIVER              SCOPE
* d076d58531f6        bridge              bridge              local
* f5bf87d4e62e        host                host                local
* df5539a0ac8f        none                null                local


Check standalone dockers are running 
! docker ps
* 293cfd15fafa        influxdb                        "/entrypoint.sh infl…"   2 days ago          Up 11 seconds       0.0.0.0:8086->8086/tcp    influxdb
* a96567852107        jupyter/scipy-notebook:latest   "tini -g -- start-no…"   6 days ago          Up 7 hours          0.0.0.0:10000->8888/tcp   jupyter


Inspect network bridge

`docker network inspect bridge`

[
    { ### bridge network information
        "Name": "bridge",
        "Id": "08db9ee2453753a66273b5ef723d4afb63c55bc59fd47bc1f6b1a56adfbfadcb",
        "Created": "2020-07-13T10:34:30.335929221-05:00",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": null,
            "Config": [
                {
                    "Subnet": "172.17.0.0/16",
                    "Gateway": "172.17.0.1"  ### this is the gateway between the host and the bridge network
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": { ## container informations
            "293cfd15fafa646ad79b27be60e2ccc950af583ba1e5c32f3ee30acde0567750": {
                "Name": "influxdb",
                "EndpointID": "03cc39c9a64d0c97e4eae317cfd3e1ff86773bca3652ffbdb0b6794bcea6bedc",
                "MacAddress": "02:42:ac:11:00:03",
                "IPv4Address": "172.17.0.3/16",
                "IPv6Address": ""
            },
            "a96567852107d54f49d756c1525ea7f90878cc138c24bf7b958d835ed5f8d952": {
                "Name": "jupyter",
                "EndpointID": "1920d239141d0a4009627337c013368c93d353a45614e0c2ffee1785e4262fae",
                "MacAddress": "02:42:ac:11:00:02",
                "IPv4Address": "172.17.0.2/16",
                "IPv6Address": ""
            }
        },
        "Options": {
            "com.docker.network.bridge.default_bridge": "true",
            "com.docker.network.bridge.enable_icc": "true",
            "com.docker.network.bridge.enable_ip_masquerade": "true",
            "com.docker.network.bridge.host_binding_ipv4": "0.0.0.0",
            "com.docker.network.bridge.name": "docker0",
            "com.docker.network.driver.mtu": "1500"
        },
        "Labels": {}
    }
]


Connect to docker images with root user

! docker exec -it influxdb /bin/bash

run `ip addr show` command in container command line to show network interfaces

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
13: eth0@if14: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default
    link/ether 02:42:ac:11:00:03 brd ff:ff:ff:ff:ff:ff link-netnsid 0
    inet 172.17.0.3/16 brd 172.17.255.255 scope global eth0   # inet address is the same as one indicated in the container list 
       valid_lft forever preferred_lft forever

Check docker container has access to internet by pinging google.com

ping -c 2 google.com

PING google.com (172.217.9.14) 56(84) bytes of data.
64 bytes from dfw28s02-in-f14.1e100.net (172.217.9.14): icmp_seq=1 ttl=112 time=42.8 ms
64 bytes from dfw28s02-in-f14.1e100.net (172.217.9.14): icmp_seq=2 ttl=112 time=38.5 ms
64 bytes from dfw28s02-in-f14.1e100.net (172.217.9.14): icmp_seq=3 ttl=112 time=45.6 ms
^C
--- google.com ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2002ms
rtt min/avg/max/mdev = 38.554/42.373/45.688/2.943 ms


# Setting user-defined network

`docker network create --driver bridge docker-net`

# to create and connect new dokcers
docker run -dit --name alpine1 --network alpine-net alpine ash

# to connect already created dockers
docker network connect bridge alpine4